<a href="https://colab.research.google.com/github/ecordeiro/cefet/blob/main/busca_aleatoria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [184]:
import numpy as np
import random

In [185]:
class Individual:
    def __init__(self, chromosome_length, limites):
        # Inicializa um indivíduo com um cromossomo aleatório
        self.fitness = None
        self.dominated_solutions = []  # Soluções dominadas por este indivíduo
        self.domination_count = 0  # Contagem de quantos indivíduos dominam este indivíduo
        self.limite_inferior, self.limite_superior = limites
        self.chromosome = [np.random.uniform(self.limite_inferior,self.limite_superior) for _ in range(chromosome_length)]
        self.crowding_distance = 0

def f1(ind):
    n = len(ind.chromosome)
    valor = sum([(elemento**2 - 4) - 10 * np.cos(2 * np.pi * (elemento**2 - 4)) for elemento in ind.chromosome])
    return 10 * n + valor

def f2(ind):
    n = len(ind.chromosome)
    valor = sum([(elemento**2 - 10 * np.cos(2 * np.pi * elemento)) for elemento in ind.chromosome])
    return 10 * n + valor

def dominancia(fitness1, fitness2):
    # Verifica se fitness1 domina fitness2
    return all(fitness1[i] <= fitness2[i] for i in range(len(fitness1))) and any(fitness1[i] < fitness2[i] for i in range(len(fitness1)))

def calcula_domininancia_populacao(p):
    # Zera as listas de soluções dominadas e contagens de dominação
    for ind in p:
        ind.dominated_solutions = []

    # Calcula as soluções dominadas e atualiza a contagem de dominação para cada indivíduo
    for i in range(len(p)):
        for j in range(i + 1, len(p)):
            if dominancia(p[i].fitness, p[j].fitness):
                p[j].dominated_solutions.append(i)
                p[i].domination_count += 1
            elif dominancia(p[j].fitness, p[i].fitness):
                p[i].dominated_solutions.append(j)
                p[j].domination_count += 1
    return p

def calcula_crowding_distance(p):
    # Calcula a distância de multidão de cada indivíduo
    for ind in p:
        ind.crowding_distance = 0

    num_objectives = len(p[0].fitness)
    # Calcula a distância de multidão para cada objetivo
    for m in range(num_objectives):
        p.sort(key=lambda x: x.fitness[m])
        p[0].crowding_distance = float('inf')
        p[-1].crowding_distance = float('inf')
        for i in range(1, len(p) - 1):
            p[i].crowding_distance += (p[i + 1].fitness[m] - p[i - 1].fitness[m])
    return p

def gerar_populacao(population_size,chromosome_length,limites):
    # population_size = 50
    # chromosome_length = 2
    # limites = (-5.12, 5.12)
    populacao = [Individual(chromosome_length, limites) for _ in range(population_size)]

    for p in populacao:
        p.fitness = [f1(p),f2(p)]

    return populacao


In [202]:
qtd_individuos = 200
tamanho_cromossomos = 2
limites = (-5.12, 5.12)

populacao = gerar_populacao(qtd_individuos,tamanho_cromossomos,limites)

In [199]:
i=2
numero_sem_solucoes_dominadas = 0
geracoes = 2
for i in range(2, 100):
    num_geracoes = i
    qtd_individuos = 50
    tamanho_cromossomos = 2
    limites = (-5.12, 5.12)

    for _ in range(num_geracoes):
        #Cria nova população
        nova_geracao = gerar_populacao(qtd_individuos,tamanho_cromossomos,limites)
        pop_total = populacao + nova_geracao
        populacao_total = calcula_domininancia_populacao(pop_total)
        populacao_total = calcula_crowding_distance(pop_total)
        populacao_total.sort(key=lambda x: (x.domination_count, -x.crowding_distance))
        populacao = populacao_total[:len(populacao)]  # Seleciona os melhores indivíduos

    non_dominated_solutions = [ind for ind in populacao if ind.domination_count == 0]
    if len(non_dominated_solutions)>numero_sem_solucoes_dominadas:
        numero_sem_solucoes_dominadas = len(non_dominated_solutions)
        geracoes = i

    print(f"Número de soluções não dominadas para a geracao{i}", len(non_dominated_solutions))


Número de soluções não dominadas para a geracao2 6
Número de soluções não dominadas para a geracao3 6
Número de soluções não dominadas para a geracao4 6
Número de soluções não dominadas para a geracao5 6
Número de soluções não dominadas para a geracao6 6
Número de soluções não dominadas para a geracao7 6
Número de soluções não dominadas para a geracao8 7
Número de soluções não dominadas para a geracao9 7
Número de soluções não dominadas para a geracao10 5
Número de soluções não dominadas para a geracao11 6
Número de soluções não dominadas para a geracao12 6
Número de soluções não dominadas para a geracao13 8
Número de soluções não dominadas para a geracao14 9
Número de soluções não dominadas para a geracao15 9
Número de soluções não dominadas para a geracao16 7
Número de soluções não dominadas para a geracao17 7
Número de soluções não dominadas para a geracao18 7
Número de soluções não dominadas para a geracao19 7
Número de soluções não dominadas para a geracao20 9
Número de soluções n

KeyboardInterrupt: 

In [207]:
num_geracoes = 1000
qtd_individuos = 50
tamanho_cromossomos = 2
limites = (-5.12, 5.12)

for _ in range(num_geracoes):
    #Cria nova população
    nova_geracao = gerar_populacao(qtd_individuos,tamanho_cromossomos,limites)
    pop_total = populacao + nova_geracao
    populacao_total = calcula_domininancia_populacao(pop_total)
    populacao_total = calcula_crowding_distance(pop_total)
    populacao_total.sort(key=lambda x: (x.domination_count, -x.crowding_distance))
    populacao = populacao_total[:len(populacao)]  # Seleciona os melhores indivíduos

non_dominated_solutions = [ind for ind in populacao if ind.domination_count == 0]
print(f"Número de soluções não dominadas para a geracao{i}", len(non_dominated_solutions))

Número de soluções não dominadas para a geracao36 15


In [208]:
import altair as alt
import pandas as pd

points = [ind.fitness for ind in populacao if ind.domination_count == 0]

# Create lists named x and y and populate them with the first and second values respectively from each sub-list in points
x = [point[0] for point in points]
y = [point[1] for point in points]

# Create a DataFrame from x and y
source = pd.DataFrame({'x': x, 'y': y})

# Create a scatter plot using x and y lists
scatter = alt.Chart(source).mark_circle().encode(
    x=alt.X('x', axis=alt.Axis(title='X')),
    y=alt.Y('y', axis=alt.Axis(title='Y')),
    tooltip = ['x','y']
).interactive()

# Create a line plot to connect the points
line = alt.Chart(source).mark_line().encode(
    x='x',
    y='y'
)

# Combine the scatter plot and line plot
chart = scatter + line

# Display the plot with the x-axis label as 'X' and the y-axis label as 'Y'
chart = chart.properties(title='Scatter Plot with Connected Points')
#chart.save('scatter_plot_with_connected_points.json')
chart.display()


alt.LayerChart(...)